## Extracting Titanic Disaster Data From Kaggle

In [1]:
#!pip install python-dotenv
#installed via anaconda-navigator in py27 virtual environment 

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
from dotenv import load_dotenv, find_dotenv
#this allows credentials in .env file to be read

In [7]:
# find .env automatically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)

True

In [8]:
# extracting environment variable using os.environ.get
import os
KAGGLE_USERNAME = os.environ.get("KAGGLE_USERNAME")
print(KAGGLE_USERNAME)

kevinmckeown


In [9]:
# imports
import requests
from requests import session
import os
from dotenv import load_dotenv, find_dotenv

In [11]:
!kaggle competitions download -c titanic
#done this from command line, using API key not password

/bin/sh: 1: kaggle: not found


In [10]:
# payload for post 
payload = {
    'action': 'login',
    'username': os.environ.get("KAGGLE_USERNAME"),
#    'password': os.environ.get("KAGGLE_PASSWORD")
    'key': os.environ.get("KAGGLE_KEY")
}

# url for train file (get the link from Kaggle website)
url = 'https://www.kaggle.com/c/titanic/download/train.csv'


# setup session
with session() as c:
    # post request
    c.post('https://www.kaggle.com/account/login', data=payload)
    # get request
    response = c.get(url)
    # print response text
    print(response.text)

<!DOCTYPE html>
<html lang="en">
<head>
    <title>Kaggle: Your Home for Data Science</title>
    <meta charset="utf-8" />
    <meta name="robots" content="index, follow" />
    <meta name="description" content="Kaggle is the world&#x2019;s largest data science community with powerful tools and resources to help you achieve your data science goals." />
    <meta name="turbolinks-cache-control" content="no-cache" />
        <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0">
    <meta name="theme-color" content="#008ABC" />
    <script nonce="QCvcyn6llG8jka0yhOSprg==" type="text/javascript">
        window["pageRequestStartTime"] = 1588843928445;  
        window["pageRequestEndTime"] = 1588843928449;
        window["initialPageLoadStartTime"] = new Date().getTime();
    </script>
    <link rel="preconnect" href="https://www.google-analytics.com" crossorigin="anonymous" /><link rel="preconnect" href="https://stats.g.doubleclick.ne

In [10]:
from requests import session
# payload
payload = {
    'action': 'login',
    'username': os.environ.get("KAGGLE_USERNAME"),
    'password': os.environ.get("KAGGLE_PASSWORD")
}


def extract_data(url, file_path):
    '''
    extract data from kaggle
    '''
    # setup session
    with session() as c:
        c.post('https://www.kaggle.com/account/login', data=payload)
        # oppen file to write
        with open(file_path, 'w') as handle:
            response = c.get(url, stream=True)
            for block in response.iter_content(1024):
                handle.write(block)


In [11]:
# urls
train_url = 'https://www.kaggle.com/c/titanic/download/train.csv'
test_url = 'https://www.kaggle.com/c/titanic/download/test.csv'

# file paths
raw_data_path = os.path.join(os.path.pardir,'data','raw')
train_data_path = os.path.join(raw_data_path, 'train.csv')
test_data_path = os.path.join(raw_data_path, 'test.csv')

# extract data
extract_data(train_url,train_data_path)
extract_data(test_url,test_data_path)

In [12]:
!ls -l ../data/raw

total 160
-rw-rw-r-- 1 kevin kevin  3258 Dec 11 01:17 gender_submission.csv
-rw-rw-r-- 1 kevin kevin 28629 Dec 11 01:17 test.csv
-rw-rw-r-- 1 kevin kevin 34877 May  7 10:35 titanic.zip
-rw-rw-r-- 1 kevin kevin 61194 Dec 11 01:17 train.csv


In [14]:
!

In [15]:
!head -n 5 ../data/raw/train.csv

In [16]:
!head -n 5 ../data/raw/test.csv

### Builiding the file script

In [21]:
import os
get_raw_data_script_file = os.path.join(os.path.pardir,'src','data','get_raw_data.py')

In [22]:
%%writefile $get_raw_data_script_file
# -*- coding: utf-8 -*-
import os
from dotenv import find_dotenv, load_dotenv
from requests import session
import logging


# payload for login to kaggle
payload = {
    'action': 'login',
    'username': os.environ.get("KAGGLE_USERNAME"),
    'password': os.environ.get("KAGGLE_PASSWORD")
}


def extract_data(url, file_path):
    '''
    method to extract data
    '''
    with session() as c:
        c.post('https://www.kaggle.com/account/login', data=payload)
        with open(file_path, 'w') as handle:
            response = c.get(url, stream=True)
            for block in response.iter_content(1024):
                handle.write(block)


                
def main(project_dir):
    '''
    main method
    '''
    # get logger
    logger = logging.getLogger(__name__)
    logger.info('getting raw data')
    
    # urls
    train_url = 'https://www.kaggle.com/c/titanic/download/train.csv'
    test_url = 'https://www.kaggle.com/c/titanic/download/test.csv'

    # file paths
    raw_data_path = os.path.join(project_dir,'data','raw')
    train_data_path = os.path.join(raw_data_path, 'train.csv')
    test_data_path = os.path.join(raw_data_path, 'test.csv')

    # extract data
    extract_data(train_url,train_data_path)
    extract_data(test_url,test_data_path)
    logger.info('downloaded raw training and test data')


if __name__ == '__main__':
    # getting root directory
    project_dir = os.path.join(os.path.dirname(__file__), os.pardir, os.pardir)
    
    # setup logger
    log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    logging.basicConfig(level=logging.INFO, format=log_fmt)

    # find .env automatically by walking up directories until it's found
    dotenv_path = find_dotenv()
    # load up the entries as environment variables
    load_dotenv(dotenv_path)

    # call the main
    main(project_dir)


Writing ..\src\data\get_raw_data.py


In [26]:
!python $get_raw_data_script_file

2017-01-22 12:05:01,280 - __main__ - INFO - getting raw data
2017-01-22 12:05:20,371 - __main__ - INFO - downloaded raw training and test data


In [27]:
!head -n 5 ../data/raw/train.csv

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked

1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S

2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C

3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S

4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S



In [28]:
!head -n 5 ../data/raw/test.csv

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked

892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,,Q

893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7,,S

894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,,Q

895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,,S

